In [6]:
import pandas as pd
import pickle
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Step 1: Load your dataset
df = pd.read_csv("new.csv")

# Step 2: Remove rows with NaN values in the 'processed_info' column
df = df.dropna(subset=['crimeaditionalinfo'])

# Step 3: Remove punctuation from the 'processed_info' column
df['crimeaditionalinfo'] = df['crimeaditionalinfo'].str.translate(str.maketrans('', '', string.punctuation))

# Step 4: Encode the target columns (category and sub_category) as numeric values
category_encoder = LabelEncoder()
sub_category_encoder = LabelEncoder()

df['category'] = category_encoder.fit_transform(df['category'])
df['sub_category'] = sub_category_encoder.fit_transform(df['sub_category'])

# Step 5: Split the data into train and test sets
X = df['crimeaditionalinfo']  # Input text
y = df[['category', 'sub_category']]  # Output labels (multi-label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # Limit to top 5000 words

# Step 7: Build the pipeline
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('clf', MultiOutputClassifier(LogisticRegression(max_iter=1000)))
])

# Step 8: Train the model
pipeline.fit(X_train, y_train)

# Step 9: Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Step 10: Calculate accuracy for both category and sub_category
category_accuracy = accuracy_score(y_test['category'], y_pred[:, 0])
sub_category_accuracy = accuracy_score(y_test['sub_category'], y_pred[:, 1])

# Step 11: Print accuracy scores
print(f"Category Accuracy: {category_accuracy * 100:.2f}%")
print(f"Sub-Category Accuracy: {sub_category_accuracy * 100:.2f}%")

# Step 12: Save the model, vectorizer, and encoders
with open('crime_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

with open('category_encoder.pkl', 'wb') as f:
    pickle.dump(category_encoder, f)

with open('sub_category_encoder.pkl', 'wb') as f:
    pickle.dump(sub_category_encoder, f)

print("Model and encoders have been saved!")


Model and encoders have been saved!
